In [1]:
from discussion_agents.cog.prompts.self_refine import (
    GSM8K_FEEDBACK_FEWSHOT_EXAMPLES,
    GSM8K_FEWSHOT_EXAMPLES,
    SELF_REFINE_FEEDBACK_INSTRUCTION_GSM8K,
    SELF_REFINE_INSTRUCTION_GSM8K,
)
from discussion_agents.cog.functional.self_refine import _prompt_agent, _prompt_feedback
from langchain_community.chat_models.openai import ChatOpenAI
import dotenv
import os

dotenv.load_dotenv("../.env")
openai_api_key = os.getenv("OPENAI_API_KEY")

llm = ChatOpenAI(openai_api_key=openai_api_key)

question = "A robe takes 2 bolts of blue fiber and half that much white fiber.  How many bolts in total does it take?"

In [2]:
examples = GSM8K_FEWSHOT_EXAMPLES
prompt = SELF_REFINE_INSTRUCTION_GSM8K

solution = _prompt_agent(
    llm=llm,
    question=question,
    examples=examples,
    prompt=prompt
)

<==============================================================>
# Q: Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did Jason give to Denny?
# solution using Python:

def solution():
    """Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did Jason give to Denny?"""
    jason_lollipops_initial = 20
    jason_lollipops_after = 12
    denny_lollipops = jason_lollipops_initial - jason_lollipops_after
    result = denny_lollipops
    return result



# Q: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?
# solution using Python:

def solution():
    """There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?"""
    trees_initial = 15
    

In [3]:
print(solution)

def solution():
    """A robe takes 2 bolts of blue fiber and half that much white fiber. How many bolts in total does it take?"""
    blue_fiber = 2
    white_fiber = blue_fiber / 2
    total_fiber = blue_fiber + white_fiber
    result = total_fiber
    return result


In [4]:
feedback_examples = GSM8K_FEEDBACK_FEWSHOT_EXAMPLES
feedback_prompt = SELF_REFINE_FEEDBACK_INSTRUCTION_GSM8K

feedback_and_improved_solution = _prompt_feedback(
    llm=llm,
    examples=feedback_examples,
    solution=solution,
    prompt=feedback_prompt
)

KeyError: 'format_instructions'

In [ ]:
print(feedback_and_improved_solution)

In [5]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain.prompts import PromptTemplate

feedback_examples = GSM8K_FEEDBACK_FEWSHOT_EXAMPLES
feedback_prompt = SELF_REFINE_FEEDBACK_INSTRUCTION_GSM8K


response_schemas = [
    ResponseSchema(name="feedback", description="the feedback to the code"),
    ResponseSchema(
        name="solution",
        description="the improved solution.",
    ),
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    template=feedback_prompt,
    input_variables=['examples', 'solution'],
    partial_variables={"format_instructions": format_instructions}
)

chain = prompt | llm | output_parser

In [9]:
out = chain.invoke({"examples": feedback_examples, "solution": solution})

In [10]:
out

{'feedback': 'The code is correct, no errors found.',
 'solution': 'def solution():\n    """A robe takes 2 bolts of blue fiber and half that much white fiber. How many bolts in total does it take?"""\n    blue_fiber = 2\n    white_fiber = blue_fiber / 2\n    total_fiber = blue_fiber + white_fiber\n    result = total_fiber\n    return result'}

In [12]:
print(out['solution'])

def solution():
    """A robe takes 2 bolts of blue fiber and half that much white fiber. How many bolts in total does it take?"""
    blue_fiber = 2
    white_fiber = blue_fiber / 2
    total_fiber = blue_fiber + white_fiber
    result = total_fiber
    return result


In [ ]:
feedback = out['feedback']
improved_solution = out['solution']
prefix = f"""{question_prefix}{solution}{intra_example_sep}{feedback_instruction}{answer_prefix}"""
gen_ans = f"""

{feedback}

{improved_solution}"""
new_example = f"{prefix}{gen_ans}"
feedback_examples = f"{feedback_examples}{new_example}"
